In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import copy

# Settings for printing numpy arrays
np.set_printoptions(precision=4, edgeitems=50, suppress=True)
np.core.arrayprint._line_width = 250

# Dual Simplex Method

Below is defined a function which comprises the Simplex Method helped by a Dual Problem which uses the table solution approach to solve Linear Programming Problems where we have to minimize or maximize an objective function under some restrictions.

In [3]:
def dual_simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, slack_indexes, itera = 1):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
    
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    ZX = np.ravel(tableau[0, 1:n_plus_m + 1])
    XBX = tableau[1:, 1:n_plus_m + 1]
    ZRHS = np.array([tableau[0, n_plus_m + 1]])
    XBRHS = (tableau[1:, n_plus_m + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n_plus_m - m)))
    
    print("\nEntry tableau")
    print(tableau)
    
    positive_b = True
    for b_i in np.ravel(XBRHS):
        if b_i < 0:
            positive_b = False
            break
    
    # Validating if we stop or not the optimization
    if positive_b:   # We stop
        
        # Printing the final results
        print("\n\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal primal solution is")
        results_primal_ls = [str(round(i, 4)) for i in b_]
        print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_primal_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        print("\nWith performance z =", perf_z)
        print("\nAnd the optimal dual solution is")
        results_dual_ls = np.ravel(-1 * tableau[0, [i + 1 for i in slack_indexes]]).tolist()
        results_dual_ls_f = [str(round(i, 4)) for i in results_dual_ls]
        print("\033[92m[ " + ', '.join(['w' + str(i + 1) for i in range(m)]) + " ] = [ " + ', '.join(results_dual_ls_f) + " ]\033[0m\n\n")
        
        indexes_variables = [int(var[1:]) for var in basic_variables + non_basic_variables]
        dict_primal_solution = dict(zip(indexes_variables, [b_i for b_i in b_]))
        sorted_dict_primal_solution = dict(sorted(dict_primal_solution.items(), key = lambda x:x[0]))
        primal_solution = np.array(list(sorted_dict_primal_solution.values()))
        
        dict_dual_solution = dict(zip([i for i in range(m)], results_dual_ls))
        sorted_dict_dual_solution = dict(sorted(dict_dual_solution.items(), key = lambda x:x[0]))
        dual_solution = np.array(list(sorted_dict_dual_solution.values()))
        
        #print(solution)
        return primal_solution, dual_solution
        
    else:   # We continue
        
        br = min(XBRHS)
        r = np.argmin(XBRHS)
        
        # Calculating and printing the y_ri's
        y_r = np.ravel(XBX[r, :])
        
        print("\nr =", r + 1, "-> row no.", r + 1, "of x_B part (", basic_variables[r], ")")
        print("y_r")
        print(y_r)
        
        # Analyzing if the Dual Problem is or not boundable by the condition y_r >= 0
        flag = True
        aux_counter = 0
        for y_i in y_r:
            if y_i >= 0:
                aux_counter += 1      
        if aux_counter == len(y_r):
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_k_ls = list()
            for i in range(n_plus_m):
                if y_r[i] < 0:
                    quot_ls.append(ZX[i] / y_r[i])
                    index_k_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_k_arr = np.array(index_k_ls)
            k = index_k_arr[np.argmin(quot_arr)]
            min_quot = min(quot_arr)
            print("\nk =", k + 1, "-> column no.", k + 1, "of x part (", variables[k], ")")
            print("min_quotient")
            print(min_quot)
            
            # Pivoting
            pivot = y_r[k]
            print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, k + 1] * tableau[r + 1, :]
            
            print("\nPivoted tableau")
            print(tableau)
            # Exchanging the indexes
            k_relative = 0
            for i in range(len(non_basic_indexes)):
                if k == non_basic_indexes[i]:
                    k_relative = i
                    break
            print("\n\033[94m", non_basic_variables[k_relative], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_indexes[r]
            basic_indexes[r] = non_basic_indexes[k_relative]
            non_basic_indexes[k_relative] = aux_vars
            
            # Recursive call
            return dual_simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, slack_indexes, itera + 1)

        else:   # Not boundable
            
            print("\n\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe Dual Problem is not boundable, thus the primal feasible region is empty\n\n")
            
            return False, False
            

In [4]:
def construct_tableau(A, b, c, basic_indexes):
    m, n_plus_m = np.shape(A)
    B = A[:m, basic_indexes]
    c_b = c[basic_indexes]
    b_ = np.linalg.inv(B) @ b

    one_array = np.ones(1, dtype=int)
    zero_array_v = np.ravel(np.zeros(m, dtype=int)).reshape(1, m)
    ZX = np.ravel((c_b @ np.linalg.inv(B) @ A) - c)
    XBX = np.linalg.inv(B) @ A
    ZRHS = np.full((1), c_b @ np.ravel(b_))
    XBRHS = b_.reshape(1, m)

    tableau = np.vstack((np.concatenate((one_array, ZX, ZRHS), axis = 0).reshape(1, n_plus_m + 2),
                  np.concatenate((zero_array_v.T, XBX, XBRHS.T), axis = 1)))
    
    return tableau

In [5]:
import itertools

def basic_indexes_dual_feasible_solution(A, b, c, indexes, m):
    all_combinations = itertools.combinations(indexes, m)
    possible_basic_indexes = list()
    for basic_indexes in all_combinations:
        try:
            basic_indexes = list(basic_indexes)
            B = A[:m, basic_indexes]
            c_b = c[basic_indexes]
            ZX = np.ravel((c_b @ np.linalg.inv(B) @ A) - c)
            
            dual_feasible = True
            for zx in ZX:
                if zx > 0:
                    dual_feasible = False
                    break
            if dual_feasible:
                possible_basic_indexes.append(basic_indexes)  
        except:
            pass
    
    return possible_basic_indexes

In [6]:
def basic_variables_dual_feasible_solution(possible_basic_indexes, variables):
    possible_basic_variables = list()
    for psb in possible_basic_indexes:
        possible_basic_variables.append([variables[i_psb] for i_psb in psb])
    
    return possible_basic_variables

The Dual Simplex Method, as well as the Simplex Method implementations previously done, needs to preprocess our initial data in order to adjust it into a a particular tableau with the below entries:

|         | $z$ | $x_{B}$ | $x_{N}$                | $RHS$         |
|:---:|:---:|:---:|:---:|:---:|
| $z$     | $1$ | $0$     | $c_{b}B^{-1}N - c_{N}$ | $c_b \bar{b}$ |
| $X_{B}$ | $0$ | $I$     | $B^{-1}N$              | $\bar{b}$     |

That if we break it down we get the following equivalent tableau

|          | $z$      | $x_1$      | $x_2$      | $...$ | $x_n$      | $x_{n+1}$         | $...$ | $x_{n+m}$         | $RHS$         |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$      | $1$      | $z_1-c_1$  | $z_2-c_2$  | $...$ | $z_n-c_n$  | $z_{n+1}-c_{n+1}$ | $...$ | $z_{n+m}-c_{n+m}$ | $c_b \bar{b}$ |
| $X_{B1}$ | $0$      | $Y_{1, 1}$ | $Y_{1, 2}$ | $...$ | $Y_{1, n}$ | $Y_{1, n+1}$      | $...$ | $Y_{1, n+m}$      | $\bar{b_1}$        |
| $X_{B2}$ | $0$      | $Y_{2, 1}$ | $Y_{2, 2}$ | $...$ | $Y_{2, n}$ | $Y_{2, n+1}$      | $...$ | $Y_{2, n+m}$      | $\bar{b_2}$          |
| $\vdots$ | $\vdots$ | $\vdots$   | $\vdots$   |       | $\vdots$   | $\vdots$          |       | $\vdots$          | $\vdots$      |
| $X_{Bm}$ | $0$      | $Y_{m, 1}$ | $Y_{m, 2}$ | $...$ | $Y_{m, n}$ | $Y_{m, n+1}$      | $...$ | $Y_{m, n+m}$      | $\bar{b_m}$          |

Taking this into account, the purpose of this method is to make the $\bar{b_i}$ less negative, and eventually turning them into positive values.

So, once we have already defined the previous function, we are able to apply it.

#### Example 1:
Solve the next linear programming problem:

$$min \hspace{0.5cm} 2x_1 + 3x_2 + 4x_3$$
$$under \hspace{0.5cm} x_1 + 2x_2 + x_3 \geq 3$$
$$\hspace{1.4cm} 2x_1 - x_2 + 3x_3 \geq 4$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

First, we introduce our slack variables $x_4, x_5$ and multiply by -1 the restrictions in order to make an identity submatrix. So, the primal problem can be rewritten as
$$min \hspace{0.5cm} 2x_1 + 3x_2 + 4x_3 + 0x_4 + 0x_5$$
$$under \hspace{0.5cm} -x_1 - 2x_2 - x_3 + x_4 + 0x_5 = -3$$
$$\hspace{1.4cm} -2x_1 + x_2 - 3x_3 + 0x_4 + x_5 = -4$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 3w_1 - 4w_2$$
$$under \hspace{0.5cm} - w_1 - 2w_2 \leq 2$$
$$\hspace{1.4cm} - 2w_1 + w_2 \leq 3$$
$$\hspace{1.4cm} - w_1 - 3w_2 \leq 4$$
$$\hspace{1.4cm} w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 + w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_4, x_5$ as the basic variables and $x_1, x_2, x_3$ as the non basic ones, we get the following elements:
\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccc | ccc}
-1 & -2 & -1 & 1 & 0\\
-2 & 1 & -3 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-3\\
-4\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccc|ccc}
2 & 3 & 4 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [7]:
A = np.matrix([[-1, -2, -1, 1, 0], [-2, 1, -3, 0, 1]])
b = np.array([-3, -4])
c = np.array([2, 3, 4, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5"]

In [8]:
basic_indexes = [3, 4]
non_basic_indexes = [0, 1, 2]
slack_indexes = [3, 4]

In [9]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [10]:
tableau

matrix([[ 1., -2., -3., -4.,  0.,  0.,  0.],
        [ 0., -1., -2., -1.,  1.,  0., -3.],
        [ 0., -2.,  1., -3.,  0.,  1., -4.]])

Note that since every $z_j - c_j \leq 0$ $\hspace{0.2cm}\forall j$, we can proceed to execute the simplex dual method.

In [11]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x4, x5, x1, x2, x3 ]


Entry tableau
[[ 1. -2. -3. -4.  0.  0.  0.]
 [ 0. -1. -2. -1.  1.  0. -3.]
 [ 0. -2.  1. -3.  0.  1. -4.]]

r = 2 -> row no. 2 of x_B part ( x5 )
y_r
[-2.  1. -3.  0.  1.]

k = 1 -> column no. 1 of x part ( x1 )
min_quotient
1.0

pivot = -2.0

Pivoted tableau
[[ 1.   0.  -4.  -1.   0.  -1.   4. ]
 [ 0.   0.  -2.5  0.5  1.  -0.5 -1. ]
 [-0.   1.  -0.5  1.5 -0.  -0.5  2. ]]

 x1 enters and  x5 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x4, x1, x5, x2, x3 ]


Entry tableau
[[ 1.   0.  -4.  -1.   0.  -1.   4. ]
 [ 0.   0.  -2.5  0.5  1.  -0.5 -1. ]
 [-0.   1.  -0.5  1.5 -0.  -0.5  2. ]]

r = 1 -> row no. 1 of x_B part ( x4 )
y_r
[ 0.  -2.5  0.5  1.  -0.5]

k = 2 -> column no. 2 of x part ( x2 )
min_quotient
1.6

pivot = -2.5

Pivoted tableau
[[ 1.   0.   0.  -1.8 -1.6 -0.2  5.6]
 [-0.  -0.   1.  -0.2 -0.4  0.2  0.4]
 [-0.   1.   0.   1.4 -0.2 -0.4  2.2]]

 x2 enters and  x4 leaves the basis


Iteration  3 
[ x_B ,  x_N ] =

(array([2.2, 0.4, 0. , 0. , 0. ]), array([1.6, 0.2]))

#### Exercise 6.1:
Solve the next linear programming problem:

$$max \hspace{0.5cm} -x_1 + 2x_2$$
$$under \hspace{0.5cm} 3x_1 + 4x_2 \leq 12$$
$$\hspace{1.4cm} 2x_1 - x_2 \geq 2$$
$$\hspace{1cm}x_1, x_2 \geq 0$$

First, we introduce our slack variables $x_3, x_4$, then multiply by -1 the objective function in order to turn the maximization problem into a minimization one, and finally multiply by -1 the first constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} x_1 - 2x_2 + 0x_3 + 0x_4$$
$$under \hspace{0.5cm} -3x_1 - 4x_2 - x_3 + 0x_4 = -12$$
$$\hspace{1.4cm} 2x_1 - x_2 + 0x_3 - x_4 = 2$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 12w_1 + 2w_2$$
$$under \hspace{0.5cm} - 3w_1 + 2w_2 \leq 1$$
$$\hspace{1.4cm} - 4w_1 - w_2 \leq -2$$
$$\hspace{1.4cm} - w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_3, x_4$ as the basic variables and $x_1, x_2$ as the non basic ones, we get the following elements:
\begin{equation} A = [N | B] = 
\left[
\begin{array}{cc | cc}
-3 & -4 & -1 & 0\\
2 & -1 & 0 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-12\\
2\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{cc|cc}
1 & -2 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [12]:
A = np.matrix([[-3, -4, -1, 0], [2, -1, 0, -1]])
b = np.array([-12, 2])
c = np.array([1, -2, 0, 0])
variables = ["x1", "x2", "x3", "x4"]

In [13]:
basic_indexes = [2, 3]
non_basic_indexes = [0, 1]
slack_indexes = [2, 3]

In [14]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [15]:
tableau

matrix([[ 1., -1.,  2.,  0.,  0.,  0.],
        [ 0.,  3.,  4.,  1.,  0., 12.],
        [ 0., -2.,  1.,  0.,  1., -2.]])

Note that since $z_j - c_j \leq 0$ condition isn't fulfilled $\forall j$, we cannot proceed to execute the simplex dual method yet, so we should look for another basis that comply the previous condition.

In [16]:
possible_basic_indexes = basic_indexes_dual_feasible_solution(A, b, c, [i for i in range(len(variables))], np.shape(A)[0])
possible_basic_variables = basic_variables_dual_feasible_solution(possible_basic_indexes, variables)
possible_basic_variables

[['x1', 'x2'], ['x2', 'x4']]

Trying with $x_2, x_4$ as the basic variables we get the following changes:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{cc | cc}
-3 & -1 & -4 & 0\\
2 & 0 & -1 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-12\\
2\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{cc|cc}
1 & 0 & -2 & 0\\
\end{array}
\right]
\end{equation}

In [17]:
basic_indexes = [1, 3]
non_basic_indexes = [0, 2]
slack_indexes = [2, 3]

In [18]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [19]:
tableau

matrix([[ 1.  , -2.5 ,  0.  , -0.5 ,  0.  , -6.  ],
        [ 0.  ,  0.75,  1.  ,  0.25,  0.  ,  3.  ],
        [ 0.  , -2.75,  0.  , -0.25,  1.  , -5.  ]])

As it can be seen, changing the basis the condition complies, so now we can proceed to execute the dual simplex method.

In [20]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x2, x4, x1, x3 ]


Entry tableau
[[ 1.   -2.5   0.   -0.5   0.   -6.  ]
 [ 0.    0.75  1.    0.25  0.    3.  ]
 [ 0.   -2.75  0.   -0.25  1.   -5.  ]]

r = 2 -> row no. 2 of x_B part ( x4 )
y_r
[-2.75  0.   -0.25  1.  ]

k = 1 -> column no. 1 of x part ( x1 )
min_quotient
0.9090909090909091

pivot = -2.75

Pivoted tableau
[[ 1.      0.      0.     -0.2727 -0.9091 -1.4545]
 [ 0.      0.      1.      0.1818  0.2727  1.6364]
 [-0.      1.     -0.      0.0909 -0.3636  1.8182]]

 x1 enters and  x4 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x2, x1, x4, x3 ]


Entry tableau
[[ 1.      0.      0.     -0.2727 -0.9091 -1.4545]
 [ 0.      0.      1.      0.1818  0.2727  1.6364]
 [-0.      1.     -0.      0.0909 -0.3636  1.8182]]


Optimality reached

The optimal primal solution is
[ x2, x1, x4, x3 ] = [ 1.6364, 1.8182, 0.0, 0.0 ]

With performance z = -1.454545454545455

And the optimal dual solution is
[ w1, w2 ] = [ 0.2727, 0.9091 ]




(array([1.8182, 1.6364, 0.    , 0.    ]), array([0.2727, 0.9091]))

#### Exercise 6.2:
Solve the next linear programming problem:

$$min \hspace{0.5cm} 2x_1 + 3x_2 + 5x_3 + 6x_4$$
$$under \hspace{0.5cm} x_1 + 2x_2 + 3x_3 + x_4 \geq 2$$
$$\hspace{1.4cm} -2x_1 + x_2 - x_3 + 3x_4 \leq -3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

First, we introduce our slack variables $x_5, x_6$ and then multiply by -1 the first constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} 2x_1 + 3x_2 + 5x_3 + 6x_4 + 0x_5 + 0x_6$$
$$under \hspace{0.5cm} - x_1 - 2x_2 - 3x_3 - x_4 + x_5 + 0x_6 = -2$$
$$\hspace{1.4cm} -2x_1 + x_2 - x_3 + 3x_4 + 0x_5 + x_6 = -3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 2w_1 - 3w_2$$
$$under \hspace{0.5cm} - w_1 - 2w_2 \leq 2$$
$$\hspace{1.4cm} - 2w_1 + w_2 \leq 3$$
$$\hspace{1.4cm} - 3w_1 - w_2 \leq 5$$
$$\hspace{1.4cm} - w_1 + 3w_2 \leq 6$$
$$\hspace{1.4cm} w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 + w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_5, x_6$ as the basic variables and $x_1, x_2, x_3, x_4$ as the non basic ones, we get the following elements:
\begin{equation} A = [N | B] = 
\left[
\begin{array}{cccc | cc}
-1 & -2 & -3 & -1 & 1 & 0\\
-2 & 1 & -1 & 3 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-2\\
-3\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{cccc | cc}
2 & 3 & 5 & 6 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [21]:
A = np.matrix([[-1, -2, -3, -1, 1, 0], [-2, 1, -1, 3, 0, 1]])
b = np.array([-2, -3])
c = np.array([2, 3, 5, 6, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

In [22]:
basic_indexes = [4, 5]
non_basic_indexes = [0, 1, 2, 3]
slack_indexes = [4, 5]

In [23]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [24]:
tableau

matrix([[ 1., -2., -3., -5., -6.,  0.,  0.,  0.],
        [ 0., -1., -2., -3., -1.,  1.,  0., -2.],
        [ 0., -2.,  1., -1.,  3.,  0.,  1., -3.]])

Note that since every $z_j - c_j \leq 0$ $\hspace{0.2cm}\forall j$, we can proceed to execute the simplex dual method.

In [25]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x5, x6, x1, x2, x3, x4 ]


Entry tableau
[[ 1. -2. -3. -5. -6.  0.  0.  0.]
 [ 0. -1. -2. -3. -1.  1.  0. -2.]
 [ 0. -2.  1. -1.  3.  0.  1. -3.]]

r = 2 -> row no. 2 of x_B part ( x6 )
y_r
[-2.  1. -1.  3.  0.  1.]

k = 1 -> column no. 1 of x part ( x1 )
min_quotient
1.0

pivot = -2.0

Pivoted tableau
[[ 1.   0.  -4.  -4.  -9.   0.  -1.   3. ]
 [ 0.   0.  -2.5 -2.5 -2.5  1.  -0.5 -0.5]
 [-0.   1.  -0.5  0.5 -1.5 -0.  -0.5  1.5]]

 x1 enters and  x6 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x5, x1, x6, x2, x3, x4 ]


Entry tableau
[[ 1.   0.  -4.  -4.  -9.   0.  -1.   3. ]
 [ 0.   0.  -2.5 -2.5 -2.5  1.  -0.5 -0.5]
 [-0.   1.  -0.5  0.5 -1.5 -0.  -0.5  1.5]]

r = 1 -> row no. 1 of x_B part ( x5 )
y_r
[ 0.  -2.5 -2.5 -2.5  1.  -0.5]

k = 2 -> column no. 2 of x part ( x2 )
min_quotient
1.6

pivot = -2.5

Pivoted tableau
[[ 1.   0.   0.   0.  -5.  -1.6 -0.2  3.8]
 [-0.  -0.   1.   1.   1.  -0.4  0.2  0.2]
 [-0.   1.   0.   1.  -1.  -0.2 -0.4  1

(array([1.6, 0.2, 0. , 0. , 0. , 0. ]), array([1.6, 0.2]))

#### Exercise 6.6:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 10x_1 + 24x_2 + 20x_3 + 20x_4 + 25x_5$$
$$under \hspace{0.5cm} x_1 + x_2 + 2x_3 + 3x_4 + 5x_5 \leq 19$$
$$\hspace{1.4cm} 2x_1 + 4x_2 + 3x_3 + 2x_4 + x_5 \leq 57$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

First, we introduce our slack variables $x_6, x_7$, then multiply by -1 the objective function in order to turn the maximization problem into a minimization one, and finally multiply by -1 the first and the second constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} - 10x_1 - 24x_2 - 20x_3 - 20x_4 - 25x_5 + 0x_6 + 0x_7$$
$$under \hspace{0.5cm} - x_1 - x_2 - 2x_3 - 3x_4 - 5x_5 - x_6 + 0x_7 = -19$$
$$\hspace{1.4cm} - 2x_1 - 4x_2 - 3x_3 - 2x_4 - x_5 + 0x_6 - x_7 = -57$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 19w_1 - 57w_2$$
$$under \hspace{0.5cm} - w_1 - 2w_2 \leq -10$$
$$\hspace{1.4cm} - w_1 - 4w_2 \leq -24$$
$$\hspace{1.4cm} - 2w_1 - 3w_2 \leq -20$$
$$\hspace{1.4cm} - 3w_1 - 2w_2 \leq -20$$
$$\hspace{1.4cm} - 5w_1 - 1w_2 \leq -25$$
$$\hspace{1.4cm} - w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, to choose the basic variables by considering that a feasible solution for the dual problem is $w = [w_1, w_2] = [4, 5]$, we have to find such variables that comply $w = c_{B}B^{-1}$, i.e. $wB = c_{B}$.

Breaking down the above, we have that our matrix A is defined as

\begin{equation} A =
\left[
\begin{array}{ccccccc}
-1 & -1 & -2 & -3 & -5 & -1 & 0\\
-2 & -4 & -3 & -2 & -1 & 0 & -1\\
\end{array}
\right]
\end{equation}

So, to find basis B we retake the condition $wB = c_{B}$, which for this case looks like:

\begin{equation} wB =
[4,5]
\left[
\begin{array}{cc}
a_{1,j_1} & a_{1,j_2}\\
a_{2,j_1} & a_{2,j_2}\\
\end{array}
\right] = 
\left[
\begin{array}{cc}
c_{j_1} & c_{j_2}\\
\end{array}
\right]
\end{equation}

where $j_1, j_2$ are indexes of the variables that comply the equation.

After trying with different values we get that $j_1 = 2$ and $j_2 = 5$. Thus we are going to consider $x_2, x_5$ as the basic variables and $x_1, x_3, x_4, x_6, x_7$ as the non basic ones. So, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccccc | cc}
-1 & -2 & -3 & -1 & 0 & -1 & -5\\
-2 & -3 & -2 & 0 & -1 & -4 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-19\\
-57\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccccc | cc}
-10 & -20 & -20 & 0 & 0 & -24 & -25\\
\end{array}
\right]
\end{equation}

In [26]:
A = np.matrix([[-1, -1, -2, -3, -5, -1, 0], [-2, -4, -3, -2, -1, 0, -1]])
b = np.array([-19, -57])
c = np.array([-10, -24, -20, -20, -25, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6", "x7"]

In [27]:
basic_indexes = [1, 4]
non_basic_indexes = [0, 2, 3, 5, 6]
slack_indexes = [5, 6]

In [28]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [29]:
tableau

matrix([[   1.    ,   -4.    ,    0.    ,   -3.    ,   -2.    ,
            0.    ,   -4.    ,   -5.    , -361.    ],
        [   0.    ,    0.4737,    1.    ,    0.6842,    0.3684,
            0.    ,   -0.0526,    0.2632,   14.    ],
        [   0.    ,    0.1053,    0.    ,    0.2632,    0.5263,
            1.    ,    0.2105,   -0.0526,    1.    ]])

Note that since every $z_j - c_j \leq 0$ $\hspace{0.2cm}\forall j$, we can proceed to execute the simplex dual method.

In [30]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x2, x5, x1, x3, x4, x6, x7 ]


Entry tableau
[[   1.       -4.        0.       -3.       -2.        0.       -4.
    -5.     -361.    ]
 [   0.        0.4737    1.        0.6842    0.3684    0.       -0.0526
     0.2632   14.    ]
 [   0.        0.1053    0.        0.2632    0.5263    1.        0.2105
    -0.0526    1.    ]]


Optimality reached

The optimal primal solution is
[ x2, x5, x1, x3, x4, x6, x7 ] = [ 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0 ]

With performance z = -361.0

And the optimal dual solution is
[ w1, w2 ] = [ 4.0, 5.0 ]




(array([ 0., 14.,  0.,  0.,  1.,  0.,  0.]), array([4., 5.]))

#### Exercise 6.14:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 20x_1 + 12x_2$$
$$under \hspace{0.5cm} 1.5x_1 + 0.5x_2 \leq 100$$
$$\hspace{1.4cm} x_1 + 0.5x_2 \leq 80$$
$$\hspace{1cm}x_1, x_2 \geq 0$$

First, we introduce our slack variables $x_3, x_4$ and then multiply by -1 the objective function in order to turn the maximization problem into a minimization one, and finally multiply by -1 the first and the second constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} -20x_1 - 12x_2 + 0x_3 + 0x_4$$
$$under \hspace{0.5cm} - 1.5x_1 - 0.5x_2 - x_3 + 0x_4 = -100$$
$$\hspace{1.4cm} - x_1 - 0.5x_2 + 0x_3 - x_4 = -80$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 100w_1 - 80w_2$$
$$under \hspace{0.5cm} - 1.5w_1 - w_2 \leq -20$$
$$\hspace{1.4cm} - 0.5w_1 - 0.5w_2 \leq -12$$
$$\hspace{1.4cm} - w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_3, x_4$ as the basic variables and $x_1, x_2$ as the non basic ones, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{cc | cc}
-1.5 & -0.5 & -1 & 0\\
-1 & -0.5 & 0 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-100\\
-80\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{cc | cc}
-20 & -12 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [31]:
A = np.matrix([[-1.5, -0.5, -1, 0], [-1, -0.5, 0, -1]])
b = np.array([-100, -80])
c = np.array([-20, -12, 0, 0])
variables = ["x1", "x2", "x3", "x4"]

In [32]:
basic_indexes = [2, 3]
non_basic_indexes = [0, 1]
slack_indexes = [2, 3]

In [33]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [34]:
tableau

matrix([[  1. ,  20. ,  12. ,   0. ,   0. ,   0. ],
        [  0. ,   1.5,   0.5,   1. ,   0. , 100. ],
        [  0. ,   1. ,   0.5,   0. ,   1. ,  80. ]])

Note that since $z_j - c_j \leq 0$ condition isn't fulfilled $\forall j$, we cannot proceed to execute the simplex dual method yet, so we should look for another basis that comply the previous condition.

In [35]:
possible_basic_indexes = basic_indexes_dual_feasible_solution(A, b, c, [i for i in range(len(variables))], np.shape(A)[0])
possible_basic_variables = basic_variables_dual_feasible_solution(possible_basic_indexes, variables)
possible_basic_variables

[['x2', 'x3'], ['x2', 'x4']]

Trying with $x_2, x_4$ as the basic variables we get the following changes:
    
\begin{equation} A = [N | B] = 
\left[
\begin{array}{cc | cc}
-1.5 & -1 & -0.5 & 0\\
-1 & 0 & -0.5 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-100\\
-80\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{cc | cc}
-20 & 0 & -12 & 0\\
\end{array}
\right]
\end{equation}

In [36]:
basic_indexes = [1, 3]
non_basic_indexes = [0, 2]
slack_indexes = [2, 3]

In [37]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [38]:
tableau

matrix([[    1. ,   -16. ,     0. ,   -24. ,     0. , -2400. ],
        [    0. ,     3. ,     1. ,     2. ,     0. ,   200. ],
        [    0. ,    -0.5,     0. ,    -1. ,     1. ,   -20. ]])

As it can be seen, changing the basis the condition complies, so now we can proceed to execute the dual simplex method.

In [39]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x2, x4, x1, x3 ]


Entry tableau
[[    1.    -16.      0.    -24.      0.  -2400. ]
 [    0.      3.      1.      2.      0.    200. ]
 [    0.     -0.5     0.     -1.      1.    -20. ]]

r = 2 -> row no. 2 of x_B part ( x4 )
y_r
[-0.5  0.  -1.   1. ]

k = 3 -> column no. 3 of x part ( x3 )
min_quotient
24.0

pivot = -1.0

Pivoted tableau
[[    1.     -4.      0.      0.    -24.  -1920. ]
 [    0.      2.      1.      0.      2.    160. ]
 [   -0.      0.5    -0.      1.     -1.     20. ]]

 x3 enters and  x4 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x2, x3, x1, x4 ]


Entry tableau
[[    1.     -4.      0.      0.    -24.  -1920. ]
 [    0.      2.      1.      0.      2.    160. ]
 [   -0.      0.5    -0.      1.     -1.     20. ]]


Optimality reached

The optimal primal solution is
[ x2, x3, x1, x4 ] = [ 160.0, 20.0, 0.0, 0.0 ]

With performance z = -1920.0

And the optimal dual solution is
[ w1, w2 ] = [ 0.0, 24.0 ]




(array([  0., 160.,  20.,   0.]), array([ 0., 24.]))

#### Exercise 6.27:
Solve the next linear programming problem:

$$max \hspace{0.5cm} - 4x_1 - 6 x_2 - 18x_3$$
$$under \hspace{0.5cm} x_1 + 0x_2 + 3x_3 \geq 3$$
$$\hspace{1.4cm} 0x_1 + x_2 + 2x_3 \geq 5$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

First, we introduce our slack variables $x_4, x_5$ and then multiply by -1 the objective function in order to turn the maximization problem into a minimization one, and finally multiply by -1 the first and the second constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} 4x_1 + 6 x_2 + 18x_3 + 0x_4 + 0x_5$$
$$under \hspace{0.5cm} - x_1 + 0x_2 - 3x_3 + x_4 + 0x_5 = -3$$
$$\hspace{1.4cm} 0x_1 - x_2 - 2x_3 + 0x_4 + x_5 = -5$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 3w_1 - 5w_2$$
$$under \hspace{0.5cm} - w_1 + 0w_2 \leq 4$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 6$$
$$\hspace{1.4cm} - 3w_1 - 2w_2 \leq 18$$
$$\hspace{1.4cm} w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 + w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_4, x_5$ as the basic variables and $x_1, x_2, x_3$ as the non basic ones, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccc | cc}
-1 & 0 & -3 & 1 & 0\\
0 & -1 & -2 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-3\\
-5\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccc | cc}
4 & 6 & 18 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [40]:
A = np.matrix([[-1, 0, -3, 1, 0], [0, -1, -2, 0, 1]])
b = np.array([-3, -5])
c = np.array([4, 6, 18, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5"]

In [41]:
basic_indexes = [3, 4]
non_basic_indexes = [0, 1, 2]
slack_indexes = [3, 4]

In [42]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [43]:
tableau

matrix([[  1.,  -4.,  -6., -18.,   0.,   0.,   0.],
        [  0.,  -1.,   0.,  -3.,   1.,   0.,  -3.],
        [  0.,   0.,  -1.,  -2.,   0.,   1.,  -5.]])

Note that since every $z_j - c_j \leq 0$ $\hspace{0.2cm}\forall j$, we can proceed to execute the simplex dual method.

In [44]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x4, x5, x1, x2, x3 ]


Entry tableau
[[  1.  -4.  -6. -18.   0.   0.   0.]
 [  0.  -1.   0.  -3.   1.   0.  -3.]
 [  0.   0.  -1.  -2.   0.   1.  -5.]]

r = 2 -> row no. 2 of x_B part ( x5 )
y_r
[ 0. -1. -2.  0.  1.]

k = 2 -> column no. 2 of x part ( x2 )
min_quotient
6.0

pivot = -1.0

Pivoted tableau
[[ 1. -4.  0. -6.  0. -6. 30.]
 [ 0. -1.  0. -3.  1.  0. -3.]
 [-0. -0.  1.  2. -0. -1.  5.]]

 x2 enters and  x5 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x4, x2, x1, x5, x3 ]


Entry tableau
[[ 1. -4.  0. -6.  0. -6. 30.]
 [ 0. -1.  0. -3.  1.  0. -3.]
 [-0. -0.  1.  2. -0. -1.  5.]]

r = 1 -> row no. 1 of x_B part ( x4 )
y_r
[-1.  0. -3.  1.  0.]

k = 3 -> column no. 3 of x part ( x3 )
min_quotient
2.0

pivot = -3.0

Pivoted tableau
[[ 1.     -2.      0.      0.     -2.     -6.     36.    ]
 [-0.      0.3333 -0.      1.     -0.3333 -0.      1.    ]
 [ 0.     -0.6667  1.      0.      0.6667 -1.      3.    ]]

 x3 enters and  x4 leaves the b

(array([0., 3., 1., 0., 0.]), array([2., 6.]))

#### Exercise 6.29:
Solve the next linear programming problem:

$$min \hspace{0.5cm} 2x_1 + 3x_2 + 5x_3 + 6x_4 $$
$$under \hspace{0.5cm} x_1 + 2x_2 + 3x_3 + x_4 \geq 2$$
$$\hspace{1.4cm} - 2x_1 + x_2 - x_3 + 3x_4 \leq -3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

First, we introduce our slack variables $x_5, x_6$ and multiply by -1 the first constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} 2x_1 + 3x_2 + 5x_3 + 6x_4 + 0x_5 + 0x_6$$
$$under \hspace{0.5cm} - x_1  - 2x_2 - 3x_3 - x_4 + x_5 + 0x_6 = -2$$
$$\hspace{1.4cm} - 2x_1 + x_2 - x_3 + 3x_4 + 0x_5 + x_6 = -3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 2w_1 - 3w_2$$
$$under \hspace{0.5cm} - w_1 - 2w_2 \leq 2$$
$$\hspace{1.4cm} - 2w_1 + w_2 \leq 3$$
$$\hspace{1.4cm} - 3w_1 - w_2 \leq 5$$
$$\hspace{1.4cm} - 4w_1 + 3w_2 \leq 6$$
$$\hspace{1.4cm} w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 + w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_5, x_6$ as the basic variables and $x_1, x_2, x_3, x_4$ as the non basic ones, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{cccc | cc}
-1 & -2 & -3 & -1 & 1 & 0\\
-2 & 1 & -1 & 3 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-2\\
-3\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{cccc | cc}
2 & 3 & 5 & 6 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [45]:
A = np.matrix([[-1, -2, -3, -1, 1, 0], [-2, 1, -1, 3, 0, 1]])
b = np.array([-2, -3])
c = np.array([2, 3, 5, 6, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

In [46]:
basic_indexes = [4, 5]
non_basic_indexes = [0, 1, 2, 3]
slack_indexes = [4, 5]

In [47]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [48]:
tableau

matrix([[ 1., -2., -3., -5., -6.,  0.,  0.,  0.],
        [ 0., -1., -2., -3., -1.,  1.,  0., -2.],
        [ 0., -2.,  1., -1.,  3.,  0.,  1., -3.]])

Note that since every $z_j - c_j \leq 0$ $\hspace{0.2cm}\forall j$, we can proceed to execute the simplex dual method.

In [49]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x5, x6, x1, x2, x3, x4 ]


Entry tableau
[[ 1. -2. -3. -5. -6.  0.  0.  0.]
 [ 0. -1. -2. -3. -1.  1.  0. -2.]
 [ 0. -2.  1. -1.  3.  0.  1. -3.]]

r = 2 -> row no. 2 of x_B part ( x6 )
y_r
[-2.  1. -1.  3.  0.  1.]

k = 1 -> column no. 1 of x part ( x1 )
min_quotient
1.0

pivot = -2.0

Pivoted tableau
[[ 1.   0.  -4.  -4.  -9.   0.  -1.   3. ]
 [ 0.   0.  -2.5 -2.5 -2.5  1.  -0.5 -0.5]
 [-0.   1.  -0.5  0.5 -1.5 -0.  -0.5  1.5]]

 x1 enters and  x6 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x5, x1, x6, x2, x3, x4 ]


Entry tableau
[[ 1.   0.  -4.  -4.  -9.   0.  -1.   3. ]
 [ 0.   0.  -2.5 -2.5 -2.5  1.  -0.5 -0.5]
 [-0.   1.  -0.5  0.5 -1.5 -0.  -0.5  1.5]]

r = 1 -> row no. 1 of x_B part ( x5 )
y_r
[ 0.  -2.5 -2.5 -2.5  1.  -0.5]

k = 2 -> column no. 2 of x part ( x2 )
min_quotient
1.6

pivot = -2.5

Pivoted tableau
[[ 1.   0.   0.   0.  -5.  -1.6 -0.2  3.8]
 [-0.  -0.   1.   1.   1.  -0.4  0.2  0.2]
 [-0.   1.   0.   1.  -1.  -0.2 -0.4  1

(array([1.6, 0.2, 0. , 0. , 0. , 0. ]), array([1.6, 0.2]))

#### Exercise 6.30:
Solve the next linear programming problem:

$$min \hspace{0.5cm} 3x_1 + 5x_2 - x_3 + 2x_4 - 4x_5 $$
$$under \hspace{0.5cm} x_1 + x_2 + x_3 + 3x_4 + x_5 \leq 6$$
$$\hspace{1.4cm} - x_1 - x_2 + 2x_3 + x_4 - x_5 \geq 3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

First, we introduce our slack variables $x_6, x_7$ and multiply by -1 the second constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} 3x_1 + 5x_2 - x_3 + 2x_4 - 4x_5 + 0x_6 + 0x_7$$
$$under \hspace{0.5cm} x_1 + x_2 + x_3 + 3x_4 + x_5 + x_6 + 0x_7 = 6$$
$$\hspace{1.4cm} x_1 + x_2 - 2x_3 - x_4 + x_5 + 0x_6 + x_7 = -3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} 6w_1 - 3w_2$$
$$under \hspace{0.5cm} w_1 + w_2 \leq 2$$
$$\hspace{1.4cm} w_1 + w_2 \leq 3$$
$$\hspace{1.4cm} w_1 - 2w_2 \leq 5$$
$$\hspace{1.4cm} 3w_1 - w_2 \leq 6$$
$$\hspace{1.4cm} 1w_1 + w_2 \leq 0$$
$$\hspace{1.4cm} 1w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 + w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_6, x_7$ as the basic variables and $x_1, x_2, x_3, x_4$ as the non basic ones, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccccc | cc}
1 & 1 & 1 & 3 & 1 & 1 & 0\\
1 & 1 & -2 & -1 & 1 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
6\\
-3\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccccc | cc}
3 & 5 & -1 & 2 & -4 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [50]:
A = np.matrix([[1, 1, 1, 3, 1, 1, 0], [1, 1, -2, -1, 1, 0, 1]])
b = np.array([6, -3])
c = np.array([3, 5, -1, 2, -4, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6", "x7"]

In [51]:
basic_indexes = [5, 6]
non_basic_indexes = [0, 1, 2, 3, 4]
slack_indexes = [5, 6]

In [52]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [53]:
tableau

matrix([[ 1., -3., -5.,  1., -2.,  4.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  1.,  3.,  1.,  1.,  0.,  6.],
        [ 0.,  1.,  1., -2., -1.,  1.,  0.,  1., -3.]])

Note that since $z_j - c_j \leq 0$ condition isn't fulfilled $\forall j$, we cannot proceed to execute the simplex dual method yet, so we should look for another basis that comply the previous condition.

In [54]:
possible_basic_indexes = basic_indexes_dual_feasible_solution(A, b, c, [i for i in range(len(variables))], np.shape(A)[0])
possible_basic_variables = basic_variables_dual_feasible_solution(possible_basic_indexes, variables)
possible_basic_variables

[['x3', 'x5'], ['x5', 'x7']]

Trying with $x_3, x_5$ as the basic variables we get the following changes:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccccc | cc}
1 & 1 & 3 & 1 & 0 & 1 & 1\\
1 & 1 & -1 & 0 & 1 & -2 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
6\\
-3\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccccc | cc}
3 & 5 & 0 & 2 & 0 & -1 & -4\\
\end{array}
\right]
\end{equation}

In [55]:
basic_indexes = [2, 4]
non_basic_indexes = [0, 1, 3, 5, 6]
slack_indexes = [5, 6]

In [56]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [57]:
tableau

matrix([[  1.    ,  -7.    ,  -9.    ,  -0.    , -10.    ,   0.    ,
          -3.    ,  -1.    , -15.    ],
        [  0.    ,  -0.    ,  -0.    ,   1.    ,   1.3333,  -0.    ,
           0.3333,  -0.3333,   3.    ],
        [  0.    ,   1.    ,   1.    ,   0.    ,   1.6667,   1.    ,
           0.6667,   0.3333,   3.    ]])

As it can be seen, changing the basis the condition complies, so now we can proceed to execute the dual simplex method.

In [58]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x3, x5, x1, x2, x4, x6, x7 ]


Entry tableau
[[  1.      -7.      -9.      -0.     -10.       0.      -3.      -1.
  -15.    ]
 [  0.      -0.      -0.       1.       1.3333  -0.       0.3333  -0.3333
    3.    ]
 [  0.       1.       1.       0.       1.6667   1.       0.6667   0.3333
    3.    ]]


Optimality reached

The optimal primal solution is
[ x3, x5, x1, x2, x4, x6, x7 ] = [ 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0 ]

With performance z = -15.0

And the optimal dual solution is
[ w1, w2 ] = [ 3.0, 1.0 ]




(array([0., 0., 3., 0., 3., 0., 0.]), array([3., 1.]))

#### Exercise 6.49:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 2x_1 + x_2 - x_3$$
$$under \hspace{0.5cm} x_1 + 2x_2 + x_3 \leq 8$$
$$\hspace{1.4cm} - x_1 + x_2 - 2x_3 \leq 4$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

First, we introduce our slack variables $x_4, x_5$ and then multiply by -1 the objective function in order to turn the maximization problem into a minimization one, and finally multiply by -1 the first and the second constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} - 2x_1 - x_2 + x_3 + 0x_4 + 0x_5 $$
$$under \hspace{0.5cm} - x_1 - 2x_2 - x_3 - x_4 + 0x_5 = -8$$
$$\hspace{1.4cm} x_1 - x_2 + 2x_3 + 0x_4 - x_5 = -4$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} -8w_1 - 4w_2$$
$$under \hspace{0.5cm} - w_1 + w_2 \leq -2$$
$$\hspace{1.4cm} - 2w_1 - w_2 \leq -1$$
$$\hspace{1.4cm} - w_1 + 2w_2 \leq 1$$
$$\hspace{1.4cm} - w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_4, x_5$ as the basic variables and $x_1, x_2, x_3$ as the non basic ones, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccc | cc}
-1 & -2 & -1 & -1 & 0\\
1 & -1 & 2 & 0 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-8\\
-4\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccc | cc}
-2 & -1 & 1 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [59]:
A = np.matrix([[-1, -2, -1, -1, 0], [1, -1, 2, 0, -1]])
b = np.array([-8, -4])
c = np.array([-2, -1, 1, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5"]

In [60]:
basic_indexes = [3, 4]
non_basic_indexes = [0, 1, 2]
slack_indexes = [3, 4]

In [61]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [62]:
tableau

matrix([[ 1.,  2.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  1.,  2.,  1.,  1.,  0.,  8.],
        [ 0., -1.,  1., -2.,  0.,  1.,  4.]])

Note that since $z_j - c_j \leq 0$ condition isn't fulfilled $\forall j$, we cannot proceed to execute the simplex dual method yet, so we should look for another basis that comply the previous condition.

In [63]:
possible_basic_indexes = basic_indexes_dual_feasible_solution(A, b, c, [i for i in range(len(variables))], np.shape(A)[0])
possible_basic_variables = basic_variables_dual_feasible_solution(possible_basic_indexes, variables)
possible_basic_variables

[['x1', 'x3'], ['x1', 'x5']]

Trying with $x_1, x_3$ as the basic variables we get the following changes:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccc | cc}
-2 & -1 & 0 & -1 & -1\\
-1 & 0 & -1 & 1 & 2\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-8\\
-4\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccc | cc}
-1 & 0 & 0 & -2 & 1\\
\end{array}
\right]
\end{equation}

In [64]:
basic_indexes = [0, 2]
non_basic_indexes = [1, 3, 4]
slack_indexes = [3, 4]

In [65]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [66]:
tableau

matrix([[  1.,   0., -12.,   0.,  -5.,  -3., -52.],
        [  0.,   1.,   5.,   0.,   2.,   1.,  20.],
        [  0.,   0.,  -3.,   1.,  -1.,  -1., -12.]])

As it can be seen, changing the basis the condition complies, so now we can proceed to execute the dual simplex method.

In [67]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x1, x3, x2, x4, x5 ]


Entry tableau
[[  1.   0. -12.   0.  -5.  -3. -52.]
 [  0.   1.   5.   0.   2.   1.  20.]
 [  0.   0.  -3.   1.  -1.  -1. -12.]]

r = 2 -> row no. 2 of x_B part ( x3 )
y_r
[ 0. -3.  1. -1. -1.]

k = 5 -> column no. 5 of x part ( x5 )
min_quotient
3.0

pivot = -1.0

Pivoted tableau
[[  1.   0.  -3.  -3.  -2.   0. -16.]
 [  0.   1.   2.   1.   1.   0.   8.]
 [ -0.  -0.   3.  -1.   1.   1.  12.]]

 x5 enters and  x3 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x1, x5, x2, x4, x3 ]


Entry tableau
[[  1.   0.  -3.  -3.  -2.   0. -16.]
 [  0.   1.   2.   1.   1.   0.   8.]
 [ -0.  -0.   3.  -1.   1.   1.  12.]]


Optimality reached

The optimal primal solution is
[ x1, x5, x2, x4, x3 ] = [ 8.0, 12.0, 0.0, 0.0, 0.0 ]

With performance z = -16.0

And the optimal dual solution is
[ w1, w2 ] = [ 2.0, 0.0 ]




(array([ 8.,  0.,  0.,  0., 12.]), array([2., 0.]))

#### Exercise 6.52:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 60x_1 + 100x_2 + 80x_3$$
$$under \hspace{0.5cm} 6x_1 + 8x_2 + 10x_3 \leq 5000$$
$$\hspace{1.4cm} 100x_1 + 150x_2 + 200x_3 \leq 60000$$
$$\hspace{1.4cm} x_1 + x_2 + x_3 \leq 500$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

First, we introduce our slack variables $x_4, x_5$ and then multiply by -1 the objective function in order to turn the maximization problem into a minimization one, and finally multiply by -1 the first and the second constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} -60x_1 - 100x_2 - 80x_3 + 0x_4 + 0x_5 + 0x_6$$
$$under \hspace{0.5cm} -6x_1 - 8x_2 - 10x_3 - x_4 + 0x_5 + 0x_6 = -5000$$
$$\hspace{1.4cm} - 100x_1 - 150x_2 - 200x_3 + 0x_4 - x_5 + 0x_6  = -60000$$
$$\hspace{1.4cm} - x_1 - x_2 - x_3 + 0x_4 + 0x_5 - x_6  = -500$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} -5000w_1 - 60000w_2 - 500w_3$$
$$under \hspace{0.5cm} - 6w_1 - 100w_2 - w_3 \leq -60$$
$$\hspace{1.4cm} - 8w_1 - 150w_2 - w_3 \leq -100$$
$$\hspace{1.4cm} - 10w_1 - 200w_2 - w_3 \leq -80$$
$$\hspace{1.4cm} - w_1 + 0w_2 + 0w_3 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 + 0w_3 \leq 0$$
$$\hspace{1.4cm} 0w_1 + 0w_2 - w_3 \leq 0$$
$$\hspace{1cm}w_1, w_2, w_3 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_4, x_5, x_6$ as the basic variables and $x_1, x_2, x_3$ as the non basic ones, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccc | ccc}
-6 & -8 & -10 & -1 & 0 & 0\\
-100 & -150 & -200 & 0 & -1 & 0\\
-1 & -1 & -1 & 0 & 0 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-5000\\
-60000\\
-500\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccc | cc}
-60 & -100 & -80 & 0 & 0 & 0\\
\end{array}
\right]
\end{equation}

#### Subsection a

In [68]:
A = np.matrix([[-6, -8, -10, -1, 0, 0], [-100, -150, -120, 0, -1, 0], [-1, -1, -1, 0, 0, -1]])
b = np.array([-5000, -60000, -500])
c = np.array([-60, -100, -80, 0, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

In [69]:
basic_indexes = [3, 4, 5]
non_basic_indexes = [0, 1, 2]
slack_indexes = [3, 4, 5]

In [70]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [71]:
tableau

matrix([[    1.,    60.,   100.,    80.,     0.,     0.,     0.,     0.],
        [    0.,     6.,     8.,    10.,     1.,     0.,     0.,  5000.],
        [    0.,   100.,   150.,   120.,     0.,     1.,     0., 60000.],
        [    0.,     1.,     1.,     1.,     0.,     0.,     1.,   500.]])

Note that since $z_j - c_j \leq 0$ condition isn't fulfilled $\forall j$, we cannot proceed to execute the simplex dual method yet, so we should look for another basis that comply the previous condition.

In [72]:
possible_basic_indexes = basic_indexes_dual_feasible_solution(A, b, c, [i for i in range(len(variables))], np.shape(A)[0])
possible_basic_variables = basic_variables_dual_feasible_solution(possible_basic_indexes, variables)
possible_basic_variables

[['x2', 'x4', 'x5'],
 ['x2', 'x4', 'x6'],
 ['x2', 'x5', 'x6'],
 ['x3', 'x4', 'x6']]

Trying with $x_2, x_4, x_5$ as the basic variables we get the following changes:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccc | ccc}
-6 & -10 & 0 & -8 & -1 & 0\\
-100 & -120 & 0& -150 & 0 & -1\\
-1 & -1 & -1 & -1 & 0 & 0\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-5000\\
-60000\\
-500\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccc | cc}
-60 & -80 & 0 & -100 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [73]:
basic_indexes = [1, 3, 4]
non_basic_indexes = [0, 2, 5]
slack_indexes = [3, 4, 5]

In [74]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [75]:
tableau

matrix([[     1.,    -40.,      0.,    -20.,      0.,      0.,   -100.,
         -50000.],
        [     0.,      1.,      1.,      1.,      0.,      0.,      1.,
            500.],
        [     0.,     -2.,      0.,      2.,      1.,      0.,     -8.,
           1000.],
        [     0.,    -50.,      0.,    -30.,      0.,      1.,   -150.,
         -15000.]])

As it can be seen, changing the basis the condition complies, so now we can proceed to execute the dual simplex method.

In [76]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x2, x4, x5, x1, x3, x6 ]


Entry tableau
[[     1.    -40.      0.    -20.      0.      0.   -100. -50000.]
 [     0.      1.      1.      1.      0.      0.      1.    500.]
 [     0.     -2.      0.      2.      1.      0.     -8.   1000.]
 [     0.    -50.      0.    -30.      0.      1.   -150. -15000.]]

r = 3 -> row no. 3 of x_B part ( x5 )
y_r
[ -50.    0.  -30.    0.    1. -150.]

k = 3 -> column no. 3 of x part ( x3 )
min_quotient
0.6666666666666666

pivot = -30.0

Pivoted tableau
[[     1.         -6.6667      0.          0.          0.         -0.6667
       0.     -40000.    ]
 [     0.         -0.6667      1.          0.          0.          0.0333
      -4.          0.    ]
 [     0.         -5.3333      0.          0.          1.          0.0667
     -18.          0.    ]
 [    -0.          1.6667     -0.          1.         -0.         -0.0333
       5.        500.    ]]

 x3 enters and  x5 leaves the basis


Iteration  2 
[ x_B ,  x_N

(array([  0.,   0., 500.,   0.,   0.,   0.]), array([0.    , 0.6667, 0.    ]))

#### Subsection b

In [77]:
A = np.matrix([[-3, -4, -5, -1, 0, 0], [-172, -246, -240, 0, -1, 0], [-1, -1, -1, 0, 0, -1]])
b = np.array([-5000, -60000, -500])
c = np.array([-120, -200, -160, 0, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

In [78]:
basic_indexes = [3, 4, 5]
non_basic_indexes = [0, 1, 2]
slack_indexes = [3, 4, 5]

In [79]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [80]:
tableau

matrix([[    1.,   120.,   200.,   160.,     0.,     0.,     0.,     0.],
        [    0.,     3.,     4.,     5.,     1.,     0.,     0.,  5000.],
        [    0.,   172.,   246.,   240.,     0.,     1.,     0., 60000.],
        [    0.,     1.,     1.,     1.,     0.,     0.,     1.,   500.]])

Note that since $z_j - c_j \leq 0$ condition isn't fulfilled $\forall j$, we cannot proceed to execute the simplex dual method yet, so we should look for another basis that comply the previous condition.

In [81]:
possible_basic_indexes = basic_indexes_dual_feasible_solution(A, b, c, [i for i in range(len(variables))], np.shape(A)[0])
possible_basic_variables = basic_variables_dual_feasible_solution(possible_basic_indexes, variables)
possible_basic_variables

[['x2', 'x4', 'x6']]

In [82]:
basic_indexes = [1, 3, 5]
non_basic_indexes = [0, 2, 4]
slack_indexes = [3, 4, 5]

In [83]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [84]:
tableau

matrix([[     1.    ,    -19.8374,     -0.    ,    -35.122 ,      0.    ,
             -0.813 ,      0.    , -48780.4878],
        [     0.    ,      0.6992,      1.    ,      0.9756,      0.    ,
              0.0041,      0.    ,    243.9024],
        [     0.    ,      0.2033,     -0.    ,      1.0976,      1.    ,
             -0.0163,      0.    ,   4024.3902],
        [     0.    ,      0.3008,      0.    ,      0.0244,      0.    ,
             -0.0041,      1.    ,    256.0976]])

As it can be seen, changing the basis the condition complies, so now we can proceed to execute the dual simplex method.

In [85]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x2, x4, x6, x1, x3, x5 ]


Entry tableau
[[     1.        -19.8374     -0.        -35.122       0.         -0.813
       0.     -48780.4878]
 [     0.          0.6992      1.          0.9756      0.          0.0041
       0.        243.9024]
 [     0.          0.2033     -0.          1.0976      1.         -0.0163
       0.       4024.3902]
 [     0.          0.3008      0.          0.0244      0.         -0.0041
       1.        256.0976]]


Optimality reached

The optimal primal solution is
[ x2, x4, x6, x1, x3, x5 ] = [ 243.9024, 4024.3902, 256.0976, 0.0, 0.0, 0.0 ]

With performance z = -48780.48780487805

And the optimal dual solution is
[ w1, w2, w3 ] = [ 0.0, 0.813, 0.0 ]




(array([   0.    ,  243.9024,    0.    , 4024.3902,    0.    ,  256.0976]),
 array([0.   , 0.813, 0.   ]))

#### Subsection c

In [86]:
A = np.matrix([[-6, -8, -10, -1, 0, 0, 0], [-100, -150, -120, 0, -1, 0, 0], [-1, -1, -1, 0, 0, -1, 0], [1, 0, 0, 0, 0, 0, -1]])
b = np.array([-5000, -60000, -500, 100])
c = np.array([-60, -100, -80, 0, 0, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6", "x7"]

In [87]:
basic_indexes = [1, 2, 3, 6]
non_basic_indexes = [0, 4, 5]
slack_indexes = [3, 4, 5, 6]

In [88]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [89]:
tableau

matrix([[     1.    ,     -6.6667,      0.    ,      0.    ,      0.    ,
             -0.6667,      0.    ,      0.    , -40000.    ],
        [     0.    ,     -0.6667,      1.    ,     -0.    ,      0.    ,
              0.0333,     -4.    ,      0.    ,     -0.    ],
        [     0.    ,      1.6667,      0.    ,      1.    ,      0.    ,
             -0.0333,      5.    ,      0.    ,    500.    ],
        [     0.    ,     -5.3333,      0.    ,      0.    ,      1.    ,
              0.0667,    -18.    ,      0.    ,      0.    ],
        [     0.    ,     -1.    ,      0.    ,      0.    ,      0.    ,
              0.    ,      0.    ,      1.    ,   -100.    ]])

In [90]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x2, x3, x4, x7, x1, x5, x6 ]


Entry tableau
[[     1.         -6.6667      0.          0.          0.         -0.6667
       0.          0.     -40000.    ]
 [     0.         -0.6667      1.         -0.          0.          0.0333
      -4.          0.         -0.    ]
 [     0.          1.6667      0.          1.          0.         -0.0333
       5.          0.        500.    ]
 [     0.         -5.3333      0.          0.          1.          0.0667
     -18.          0.          0.    ]
 [     0.         -1.          0.          0.          0.          0.
       0.          1.       -100.    ]]

r = 4 -> row no. 4 of x_B part ( x7 )
y_r
[-1.  0.  0.  0.  0.  0.  1.]

k = 1 -> column no. 1 of x part ( x1 )
min_quotient
6.666666666666643

pivot = -1.0

Pivoted tableau
[[     1.          0.          0.          0.          0.         -0.6667
       0.         -6.6667 -39333.3333]
 [     0.          0.          1.         -0.          0.          0.0

(array([100.    ,  66.6667, 333.3333, 533.3333,   0.    ,   0.    ,
          0.    ]),
 array([ 0.    ,  0.6667, -0.    ,  6.6667]))